In [ ]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = 'pf-henry-405314' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

# Librerias

In [ ]:
import pandas as pd

import datetime

from google.cloud import storage
from google.cloud import bigquery
import pandas
import pytz

from google.colab import drive
from google.colab import auth

from pandas import json_normalize

from textblob import TextBlob


# Carga en bigquery de reviews unificada con sitios y atributos

In [ ]:
job = client.get_job('bquxjob_2bac3aaf_18c1b5f15d7') # Job ID inserted based on the query results selected to explore
reviews = job.to_dataframe()

In [ ]:
job = client.get_job('bquxjob_575db311_18c1b8ecb4d') # Job ID inserted based on the query results selected to explore
sitios_at = job.to_dataframe()

In [ ]:
df_unificado = reviews.merge(sitios_at, on='gmap_id')

In [ ]:
df_unificado['state'].value_counts()

Florida           431494
California        285878
Texas             264065
North Carolina    208260
Arizona           157347
Georgia           153780
Tennessee         139535
South Carolina    113006
Alabama            82170
Oklahoma           75147
Louisiana          73184
Arkansas           48565
New Mexico         41129
Mississippi        40512
Name: state, dtype: int64

In [ ]:
df_unificado.columns

Index(['user_id', 'name_x', 'time', 'rating', 'text', 'resp', 'gmap_id',
       'state', 'name_y', 'address', 'latitude', 'longitude', 'category',
       'avg_rating', 'num_of_reviews', 'Service0', 'Service1', 'Service2',
       'Accessibility0', 'Offerings0', 'Offerings1', 'Offerings2',
       'Amenities0', 'Atmosphere0', 'Crowd0'],
      dtype='object')

In [ ]:
auth.authenticate_user()
print('Authenticated')

# Especificar el proyecto y el dataset de BigQuery
project_id = 'pf-henry-405314'

table_id = 'pf-henry-405314.Unificados.maestro-reviews-sitios'

# Crear un cliente de BigQuery
client = bigquery.Client(project=project_id)

# Cargar el DataFrame a BigQuery
job_config =  bigquery.LoadJobConfig( schema =  [
                bigquery.SchemaField('user_id',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('name_x',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('time',	bigquery.enums.SqlTypeNames.DATETIME),
                bigquery.SchemaField('rating',	bigquery.enums.SqlTypeNames.INTEGER),
                bigquery.SchemaField('text',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('resp',	bigquery.enums.SqlTypeNames.INTEGER),
                bigquery.SchemaField('gmap_id',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('state',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('name_y',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('address',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('latitude',	bigquery.enums.SqlTypeNames.FLOAT),
                bigquery.SchemaField('longitude',	bigquery.enums.SqlTypeNames.FLOAT),
                bigquery.SchemaField('category',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('avg_rating',	bigquery.enums.SqlTypeNames.FLOAT),
                bigquery.SchemaField('num_of_reviews',	bigquery.enums.SqlTypeNames.INTEGER),
                bigquery.SchemaField('Service0',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Service1',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Service2',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Accessibility0',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Offerings0',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Offerings1',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Offerings2',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Amenities0',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Atmosphere0',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Crowd0',	bigquery.enums.SqlTypeNames.STRING),
                ],
              write_disposition="WRITE_TRUNCATE",)

job = client.load_table_from_dataframe(df_unificado, table_id, job_config=job_config)  # llamada a la API

job.result()  # Espera mientras se ejecuta el trabajo

table = client.get_table(table_id)  # llamada a la API

print("Cargadas {} filas y {} columnas a {}".format(table.num_rows, len(table.schema), table_id))

Authenticated
Cargadas 2114072 filas y 25 columnas a pf-henry-405314.Unificados.maestro-reviews-sitios


In [ ]:
from textblob import TextBlob

def clasificar_sentimiento(polaridad):
    if polaridad > 0.2:
        return '2'
    elif polaridad < -0.2:
        return '0'
    else:
        return '1'

comentario = "Not a good flavor and overpriced"

sentimiento = TextBlob(comentario).sentiment.polarity
clasificacion = clasificar_sentimiento(sentimiento)

print(clasificacion)


0


In [ ]:
import pandas as pd
from textblob import TextBlob

def clasificar_sentimiento(polaridad):
    if polaridad > 0.2:
        return 2
    elif polaridad < -0.2:
        return 0
    else:
        return 1

# Crear DataFrame de ejemplo
df = pd.DataFrame({'reviews': ['Not a good flavor and overpriced',
                               'The product is okay but nothing special',
                               'I love this product, it works great']})

# Aplicar análisis de sentimiento y clasificación a la columna "reviews"
df['sentimiento'] = df['reviews'].apply(lambda x: TextBlob(x).sentiment.polarity).apply(clasificar_sentimiento)

print(df)


# analisis de sentimiento

In [ ]:
job = client.get_job('bquxjob_6e83b41a_18c1c798435') # Job ID inserted based on the query results selected to explore
maestro = job.to_dataframe()
maestro

,user_id,name_x,time,rating,text,resp,gmap_id,state,name_y,address,...,Service0,Service1,Service2,Accessibility0,Offerings0,Offerings1,Offerings2,Amenities0,Atmosphere0,Crowd0
0,,Lisa Martin,2021-01-14 20:50:15.078,5,Fresh and tasty!,0,0x864c1ffaede28457:0xad75ecbeb953bd83,Texas,Pokeworks,"Pokeworks, 746 S Central Expy #110, Richardson...",...,No-contact delivery,Delivery,Takeout,Wheelchair accessible entrance,Healthy options,Organic dishes,Quick bite,Good for kids,Casual,College students
1,,Audrey Murray,2019-02-27 08:36:37.089,2,Didn't care for them...cold,0,0x864edb0330201193:0xd670bf92ed8809cc,Texas,IHOP,"IHOP, 111 S, I-45, Ennis, TX 75119",...,Curbside pickup,Delivery,Takeout,Wheelchair accessible entrance,Coffee,Happy hour drinks,Healthy options,Good for kids,Casual,College students
2,,Matthew Johnson,2018-02-07 21:05:33.669,5,Wonderful little hole in the wall kinda place,1,0x87cc6ba9970e6581:0x7f417759d9f8231b,Arkansas,Fox & Fork,"Fox & Fork, 117 S College Ave, Clarksville, AR...",...,Takeout,Delivery,No,Wheelchair accessible parking lot,Beer,Organic dishes,Wine,Good for kids,No,No
3,,Andrew Hatter,2019-09-15 16:27:10.024,5,"Phenomenal ""Tito's Bloody Mary"" and Eggs Bened...",0,0x885c24dc8e18fba9:0x6342205c7fe2e255,Tennessee,O'Charley’s Restaurant & Bar,"O'Charley’s Restaurant & Bar, 8077 Kingston Pi...",...,Outdoor seating,Curbside pickup,No-contact delivery,Wheelchair-accessible car park,Comfort food,Happy-hour food,Healthy options,Good for kids,Casual,Groups
4,,Lisa Nowak,2018-11-01 12:18:57.812,2,"Server was efficient with our needs, but the f...",0,0x885c24dc8e18fba9:0x6342205c7fe2e255,Tennessee,O'Charley’s Restaurant & Bar,"O'Charley’s Restaurant & Bar, 8077 Kingston Pi...",...,Outdoor seating,Curbside pickup,No-contact delivery,Wheelchair-accessible car park,Comfort food,Happy-hour food,Healthy options,Good for kids,Casual,Groups
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2114067,,Kevin Johnson (KJcanhelp),2020-12-20 23:10:25.875,5,First time securing the bag here and the wings...,0,0x88e7835deb373a1f:0x2fb0efa0db6dfeb1,Florida,Wingstop,"Wingstop, 1039 S Dillard St, Winter Garden, FL...",...,No-contact delivery,Delivery,Takeout,Wheelchair accessible entrance,Comfort food,Late-night food,Quick bite,Good for kids,Casual,Groups
2114068,,Shonique Smith,2020-03-06 03:20:35.248,5,Very good\nNice staff,0,0x86de3e09dbdf555d:0xd6babdf9db3b9dba,New Mexico,Valley Pizza,"Valley Pizza, 3100 W Picacho Ave, Las Cruces, ...",...,Outdoor seating,Curbside pickup,No-contact delivery,Wheelchair accessible entrance,Comfort food,Late-night food,Quick bite,Good for kids,Casual,College students
2114069,,Alosh GH,2021-05-26 16:01:55.397,5,Good,0,0x8854206d09514cb3:0xa6bf09b14dd162e4,North Carolina,Hunan Express Chinese Restaurant,"Hunan Express Chinese Restaurant, 901 N Wendov...",...,No-contact delivery,Delivery,Takeout,No,Happy hour drinks,Late-night food,Quick bite,Good for kids,Casual,No
2114070,,Coolman Coolman (Master Lee),2021-06-01 17:39:40.291,3,At 12 noon ordering a shake but they cleaning ...,1,0x86213de93e423dff:0x5f6e709ff7323f06,Louisiana,Rally's,"Rally's, 400 N Canal Blvd, Thibodaux, LA 70301",...,Curbside pickup,Delivery,Drive-through,No,Kids' menu,Late-night food,Quick bite,Good for kids,Casual,No


In [ ]:
def clasificar_sentimiento(polaridad):
    if polaridad > 0.2:
        return 2
    elif polaridad < -0.2:
        return 0
    else:
        return 1

# Aplicar análisis de sentimiento y clasificación a la columna "reviews"
maestro['sentiment'] = maestro['text'].apply(lambda x: TextBlob(x).sentiment.polarity).apply(clasificar_sentimiento)

In [ ]:
maestro.drop(columns=['user_id','name_x','address','text'], inplace=True)

In [ ]:
# Dividir la cadena en función de la coma y seleccionar la primera parte
maestro['category'] = maestro['category'].str.split(',').str[0]

In [ ]:
maestro.columns

Index(['time', 'rating', 'resp', 'gmap_id', 'state', 'name_y', 'latitude',
       'longitude', 'category', 'avg_rating', 'num_of_reviews', 'Service0',
       'Service1', 'Service2', 'Accessibility0', 'Offerings0', 'Offerings1',
       'Offerings2', 'Amenities0', 'Atmosphere0', 'Crowd0', 'sentiment'],
      dtype='object')

In [ ]:
maestro = maestro[['state','name_y', 'category', 'gmap_id', 'latitude', 'longitude', 'time', 'rating', 'sentiment', 'resp', 'avg_rating', 'num_of_reviews', 'Service0',
       'Service1', 'Service2', 'Accessibility0', 'Offerings0', 'Offerings1', 'Offerings2', 'Amenities0', 'Atmosphere0', 'Crowd0']]

In [ ]:
maestro.columns

Index(['state', 'name_y', 'category', 'gmap_id', 'latitude', 'longitude',
       'time', 'rating', 'sentiment', 'resp', 'avg_rating', 'num_of_reviews',
       'Service0', 'Service1', 'Service2', 'Accessibility0', 'Offerings0',
       'Offerings1', 'Offerings2', 'Amenities0', 'Atmosphere0', 'Crowd0'],
      dtype='object')

In [ ]:
maestro = maestro[maestro['category'].str.contains('restaurants|restaurante|restaurant|food|takeaway|lunch|deli|donut|dinner|bakery', case=False, na=False)]

In [ ]:
auth.authenticate_user()
print('Authenticated')

# Especificar el proyecto y el dataset de BigQuery
project_id = 'pf-henry-405314'

table_id = 'pf-henry-405314.Unificados.maestro-disponible'

# Crear un cliente de BigQuery
client = bigquery.Client(project=project_id)

# Cargar el DataFrame a BigQuery
job_config =  bigquery.LoadJobConfig( schema =  [
                bigquery.SchemaField('state',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('name_y',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('category',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('gmap_id',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('latitude',	bigquery.enums.SqlTypeNames.FLOAT),
                bigquery.SchemaField('longitude',	bigquery.enums.SqlTypeNames.FLOAT),
                bigquery.SchemaField('time',	bigquery.enums.SqlTypeNames.DATETIME),
                bigquery.SchemaField('rating',	bigquery.enums.SqlTypeNames.INTEGER),
                bigquery.SchemaField('sentiment',	bigquery.enums.SqlTypeNames.INTEGER),
                bigquery.SchemaField('resp',	bigquery.enums.SqlTypeNames.INTEGER),
                bigquery.SchemaField('avg_rating',	bigquery.enums.SqlTypeNames.FLOAT),
                bigquery.SchemaField('num_of_reviews',	bigquery.enums.SqlTypeNames.INTEGER),
                bigquery.SchemaField('Service0',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Service1',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Service2',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Accessibility0',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Offerings0',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Offerings1',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Offerings2',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Amenities0',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Atmosphere0',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('Crowd0',	bigquery.enums.SqlTypeNames.STRING),
                ],
              write_disposition="WRITE_TRUNCATE",)

job = client.load_table_from_dataframe(maestro, table_id, job_config=job_config)  # llamada a la API

job.result()  # Espera mientras se ejecuta el trabajo

table = client.get_table(table_id)  # llamada a la API

print("Cargadas {} filas y {} columnas a {}".format(table.num_rows, len(table.schema), table_id))

Authenticated
Cargadas 1753169 filas y 22 columnas a pf-henry-405314.Unificados.maestro-disponible
